<a href="https://colab.research.google.com/github/KhatijaBegum27/AI-ML-Projects/blob/main/GAN_FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
import gradio as gr
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# Check for GPU availability
device = "/gpu:0" if tf.config.list_physical_devices('GPU') else "/cpu:0"
print(f"Using device: {device}")

Using device: /cpu:0


In [3]:
# Fashion-MNIST class labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [4]:
def load_and_preprocess_data():
    """Load and preprocess Fashion-MNIST dataset"""
    # Load Fashion-MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

    # Normalize pixel values to 0-1 range
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Reshape for CNN (add channel dimension)
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

    # Convert labels to categorical
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    return (x_train, y_train), (x_test, y_test)

In [5]:
def create_model():
    """Create CNN model for Fashion-MNIST classification"""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [6]:
def train_model():
    """Train the Fashion-MNIST classifier"""
    print("Loading and preprocessing data...")
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()

    print("Creating model...")
    model = create_model()

    print("Training model...")
    history = model.fit(x_train, y_train,
                       batch_size=128,
                       epochs=10,
                       validation_data=(x_test, y_test),
                       verbose=1)

    # Evaluate model
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test accuracy: {test_accuracy:.4f}")

    return model, history

In [7]:
def preprocess_uploaded_image(img: Image.Image):
    """Preprocess uploaded image for prediction"""
    # Convert to grayscale if needed
    if img.mode != 'L':
        img = img.convert('L')

    # Resize to 28x28
    img = img.resize((28, 28))

    # Convert to numpy array and normalize
    img_array = np.array(img).astype('float32') / 255.0

    # Reshape for model input
    img_array = img_array.reshape(1, 28, 28, 1)

    return img_array

In [8]:
def classify_fashion_item(img: Image.Image):
    """Classify uploaded fashion item image"""
    try:
        # Preprocess the image
        processed_img = preprocess_uploaded_image(img)

        # Make prediction
        predictions = trained_model.predict(processed_img)
        predicted_class_idx = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class_idx]

        # Get class name
        predicted_class = class_names[predicted_class_idx]

        # Create results dictionary for Gradio
        results = {}
        for i, class_name in enumerate(class_names):
            results[class_name] = float(predictions[0][i])

        return results

    except Exception as e:
        return f"Error processing image: {str(e)}"


In [9]:
# Train the model
print("Starting training...")
trained_model, training_history = train_model()

Starting training...
Loading and preprocessing data...
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Creating model...
Training model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 103ms/step - accuracy: 0.5915 - loss: 1.1199 - val_accuracy: 0.8213 - val_loss: 0.4903
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 45s 96ms/step - accuracy: 0.8098 - loss: 0.5272 - val_accuracy: 0.8562 - val_loss: 0.3967
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 98ms/step - accuracy: 0.8453 - loss: 0.4374 - val_accuracy: 0.8673 - val_loss: 0.3711
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 97ms/step - accuracy: 0.8608 - loss: 0.3965 - val_accuracy: 0.8775 - val_loss: 0.3318
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 47s 100ms/step - accuracy: 0.8731 - loss: 0.3640 - val_accuracy: 0.8867 - val_loss: 0.3127
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 45s 96ms/step - accuracy: 0.8819 - loss: 0.3396 - val_accuracy: 0.8937 - val_loss: 0.2943
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 88s 108ms/step - accuracy: 0.8888 - loss: 0.3258 - val_accuracy: 0.8974 - val_loss: 0.2877
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 47s 100ms/step - accuracy: 0.8963 - loss: 0.301

In [10]:
# Create Gradio interface
iface = gr.Interface(
    fn=classify_fashion_item,
    inputs=gr.Image(type="pil", label="Upload Fashion Item Image"),
    outputs=gr.Label(num_top_classes=10, label="Classification Results"),
    title="Fashion-MNIST Classifier",
    description="Upload an image of a fashion item (clothing, shoes, bags) and get classification results. The model is trained on Fashion-MNIST dataset which includes: T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, and Ankle boot.",
    theme=gr.themes.Soft()
)

In [11]:
# Launch the interface
if __name__ == "__main__":
    iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b998ce6aab2a3f44c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
